In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
pi0scaling = {"val": 0, "suffix": "", "title": "No pi0 Scaling"} #no scaling
#pi0scaling = {"val": 1, "suffix": "_pi0flat", "title": "0.759 pi0 Scaling"} #flat scaling
#pi0scaling = {"val": 2, "suffix": "_pi0e040", "title": r"$\pi^0$ scaling: [$1 - 0.40 \times E_\pi$]"} #energy-dependent


In [ ]:
from matplotlib import pyplot as plt

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
plt.rcParams.update(params)

from load_data_run123_Dell import *
from unblinding_far_sideband import *

In [ ]:
import sys
import localSettings as ls
print(ls.main_path)
print(ls.ntuple_path)
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
import scipy.stats

In [ ]:
#which_sideband = ['np_far'       ,"farsideband", 0] # far-sideband 1eNp datasets
#which_sideband = ['0p_far'       ,"farsideband", 0] # far-sideband 1eNp datasets
#which_sideband = ['numu',"numu", 0] # numu NTuples [v41]
which_sideband = ['2plus_showers',"farsideband", 0] # far-sideband 2+ shower datasets
#which_sideband = ['np_sb_comb',"farsideband", 0] # combined far-sideband 1eNp and 2+ shower datasets
#which_sideband = ['opendata'     ,'opendata', 0]    # open datasets (5e19 / 1e19)
#which_sideband = ['opendata'     ,'fakeset9', 9]    # set9=NuWro fake dataset
#which_sideband = ['opendata'     ,'opendata', 0]    # full data

In [ ]:
samples = load_data_run123(which_sideband=which_sideband[0],
                           return_plotter=False,
                           pi0scaling=pi0scaling["val"],
                           loadpi0variables=True,
                           loadtruthfilters=True,
                           loadshowervariables=True,
                           loadnumuntuples=False,
                           loadnumuvariables=False,
                           loadfakedata=which_sideband[2],
                           USEBDT=False,
                           loadeta=False,
                           loadsystematics=True,
                           loadrecoveryvars=False,
                           loadccncpi0vars=True)

In [ ]:
import plotter
import unblinding_far_sideband
import importlib
importlib.reload(plotter)
importlib.reload(unblinding_far_sideband)

In [ ]:
#remove lee from samples!
del samples['lee']
plotter_runbyrun = {}
for run in [1, 2, 3, 123, 12]:
    this_weights, this_pot = get_weights(run,dataset=which_sideband[1])
    #remove lee from weights too!
    del this_weights['lee']
    plotter_runbyrun[run] = plotter.Plotter(samples, this_weights, pot=this_pot)
    #print(this_weights,this_pot)
    if (run==123): print(this_weights,this_pot)

In [ ]:
from unblinding_far_sideband import *

DETSYS = {'mc':False,'nue':False,'ccpi0':False,'ncpi0':False,'dirt':False,\
          'ncnopi':False,'nccpi':False,'ncpi0':False}

# choose sideband, preselection, and selection
# find defintions in unblinding_far_sideband.py
sdb_key = 'TwoPShr'
#pre_key = 'NUE'
pre_key = 'PI0'
#sel_key = 'CCNCPI0'
sel_key = 'CCPI0'
#sel_key = 'NCPI0'
run = 123

sideband = sideband_categories[sdb_key]
preselection = preselection_categories[pre_key]
sel =  selection_categories[sel_key]

DETSYSPATH = ls.ntuple_path+'/detsys/{}_{}_{}/'.format(sideband['dir'], preselection['dir'], sel['dir'])

QUERY = ""
for query in [sideband['query'],preselection['query'],sel['query']]:
    if query is None: continue
    if QUERY: QUERY += ' and '
    QUERY += query
    if run in [1, 2, 3, 12]:
        QUERY += f' and run{run}==True'
#print(QUERY)        

QUERY_pionVeto = QUERY + " and mip_mult==1"

QUERY_lead = QUERY_pionVeto + " and pi0_energy1_Y > 40 and pi0_dot1 > 0.9 and ((pi0_radlen1 >2 and  pi0_radlen1 < 80) or (pi0_radlen1 < 2 and pi0_tkfit_dedx_max_1 > 2.5))"

QUERY_sublead =  QUERY_lead + " and pi0_energy2_Y > 10 and ((pi0_radlen2 > 1) or (pi0_radlen2 < 1 and pi0_tkfit_dedx_max_2 > 2.5))"

QUERY_mass = QUERY_sublead + " and pi0_mass_Y_corr > 50 and pi0_mass_Y_corr < 180 and contained==1"

print(QUERY_mass)

QUERY_exiting1 = QUERY_mass + " and contained==1"
QUERY_contained1 = QUERY_mass + " and contained==0"

#formatting can be improved..
TITLE = ""
for text in [f'Run {run}',' - ',sideband['title'],' - ',preselection['title'],'\n',sel['title'],' - ',pi0scaling["title"]]:
    if text is None: continue
    TITLE += text



In [ ]:
VARIABLE, XTIT, BINS = 'pi0momanglecos','pi0 CosTheta',np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0])
VARIABLE, labl, XTIT , BINS = 'muon_momentum_ALL', 'Muon Momentum', r"muon momentum", np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5])
#VARIABLE, labl, XTIT , BINS = 'muon_angle', 'Muon Angle', r'Muon CosTheta', np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
VARIABLE, labl, XTIT , BINS = 'muon_momentum_contained_50', 'Contained Muon Momentum', r"muon momentum contained", np.array([0.0,0.151,0.34,0.59,0.95,1.45,3.0])
VARIABLE, labl, XTIT , BINS = 'muon_theta_contained_50', 'Contained Muon Angle', r'Muon CosTheta', np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0]) #np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
#VARIABLE, BINS, XTIT = 'muon_momentum_exiting',np.array([0.0,0.151,0.34,0.59,0.95,1.45,3.0]),r"muon momentum exiting"
#VARIABLE, labl, XTIT , BINS = 'muon_theta_exiting', 'Exiting Muon Angle', r'Muon CosTheta exiting', np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0])
VARIABLE, labl, XTIT , BINS = 'muon_angle', 'Muon Angle', r'Muon CosTheta', np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0]) #np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
VARIABLE, XTIT, BINS = 'pi0momentum', 'pi0 momentum [GeV/c]',np.array([0.0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.799])
#VARIABLE, BINS, RANGE, XTIT = 'trk_len_v',100,(0,100),"no. of tracks"
VARIABLE, BINS, RANGE, XTIT = 'muon_length_exiting',100,(0,1000),"exiting muon track length"
VARIABLE, BINS, XTIT = 'muon_momentum_exiting',np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5]),r"muon momentum exiting"
VARIABLE, BINS, XTIT = 'muon_momentum_ALL',np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5]),r"muon momentum"
#VARIABLE, BINS, RANGE, XTIT = 'muon_length_exiting',10,(0,100),"exiting muon track length"
VARIABLE, BINS, XTIT = 'muon_momentum_ALL',np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5]),r"muon momentum"
VARIABLE, labl, XTIT , BINS = 'muon_angle', 'Muon Angle', r'Muon CosTheta', np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
VARIABLE, BINS, RANGE, XTIT = 'mip_mult',5,(0,5),"no. of MIP tracks"
VARIABLE, BINS, RANGE, XTIT = 'pi0_energy1_Y', 50, (30,500),"leading shower collection-plane energy [MeV]"
VARIABLE, BINS, RANGE, XTIT = 'pi0_radlen1',50,(0,100),"leading $\gamma$ shower conversion distance [cm]"
VARIABLE, BINS, RANGE, XTIT = 'pi0_radlen2',50,(0,100),"subleading $\gamma$ shower conversion distance [cm]"
#VARIABLE, BINS, RANGE, XTIT = 'pi0_energy1_Y', 50, (30,500),"leading shower collection-plane energy [MeV]"
VARIABLE, BINS, RANGE, XTIT = 'pi0_energy2_Y', 50, (0,500),"sub-leading shower collection-plane energy [MeV]"
#VARIABLE, BINS, RANGE, XTIT = 'pi0_dot1', 20, (-1,1),"leading shower angle"
#VARIABLE, BINS, RANGE, XTIT = 'pi0_dot2', 20, (-1,1),"sub-leading shower angle"
VARIABLE, BINS, RANGE, XTIT = 'pi0_mass_Y_corr',40,(0,400),r"corrected $M_{\gamma\gamma}$ mass Y plane [MeV]"
VARIABLE, labl, XTIT , BINS = 'muon_angle', 'Muon Angle', r'Muon CosTheta', np.array([-1.,-0.50,0.0,0.30,0.5,0.65,0.78,0.90,1.0]) #np.array([-1.,-0.50,0.0,0.30,0.5,0.65,0.80,0.90,1.0]) np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
#VARIABLE, labl, XTIT , BINS = 'muon_momentum_ALL', 'Muon Momentum', r"muon momentum", np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5])
#VARIABLE, XTIT, BINS = 'pi0momanglecos','pi0 CosTheta',np.array([-1.,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0]) #np.array([-1.,-0.50,-0.25,0.0,0.25,0.5,0.63, 0.765,0.865,0.935,1.0]) #np.array([-1.,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0])
#VARIABLE, XTIT, BINS = 'pi0momentum', 'pi0 momentum [GeV/c]',np.array([0.0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.799])
#VARIABLE, labl, XTIT , BINS = 'muon_momentum_contained_50', 'Muon Momentum', r"muon momentum",np.array([0.0,0.200,0.30,0.44,0.60,0.75,0.95, 1.2, 2.5]) #np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5])
VARIABLE, labl, XTIT , BINS = 'muon_momentum_exiting', 'Muon Momentum', r"muon momentum",np.array([0.0,0.200,0.30,0.44,0.60,0.75,0.95,1.2,2.5]) #np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5])
#VARIABLE, labl, XTIT , BINS = 'mu_pi0_angle', 'Muon pion opening angle', r"muon pion opening angle", np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0]) #np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5])
#VARIABLE, BINS, XTIT = 'muon_momentum_ALL',np.array([0.0,0.200,0.30,0.44,0.60,0.75,0.95,1.20,2.0]),r"muon momentum"

#ACCEPTANCE = 'isVtxFiducial_fixed == 1 and npi0 == 1 and ccnc==0 and npion==0 and nu_pdg==14'

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARIABLE,   
    query= QUERY_mass, #QUERY_exiting, #QUERY_CONTAINED_50, #QUERY_, #QUERY_1,
    kind="ccncpi0_category",
    #kind="paper_category_xsec",
    #kind="interaction",
    draw_sys=True,
    detsysdict=DETSYS,
    DETSYSPATH=DETSYSPATH,
    COVMATRIX="",#'cov_matrix_Enu_reco_1eNp.txt',
    #genieweight = "weightSpline",
    ratio=True,
    stacksort=3,
    title=XTIT,
    bins=BINS,
    range=None,
    chisq=False,
    #asymErrs=True
)[0:3]

#print('%s %.4f %.4f %.4f' % (VARIABLE, plotter_runbyrun[run].stats['pvaluestatonly'], plotter_runbyrun[run].stats['pvaluediag'], plotter_runbyrun[run].stats['pvalue']))


#ax1.set_title(TITLE, loc='left')
#ax1.set_ylim(0, ax1.get_ylim()[1]*2.0)
#ax1.set_ylim(0., 40)
#ax1.set_yscale('log')

#plt.tight_layout()

# fix position of chi2 wrt legend
#plt.draw()
#ax1.vlines(x=135.0, ymin = 0.0, ymax = 800.0, colors='r')
ax1.set_ylim(0, ax1.get_ylim()[1]*2.2)
ax1.set_title(TITLE, loc='left')
'''legleft = (ax1.get_legend().get_window_extent().x0 < 100)
if legleft:
    for a in ax1.get_children(): 
        if len(a.findobj(match=plt.Text))==0: continue
        txt = a.findobj(match=plt.Text)[0]
        if 'n.d.f.' not in txt.get_text(): continue
        #print(txt.get_text())
        txt.set_x(0.80)'''
# END - fix position of chi2 wrt legend

#fig.show()
#save_path = ls.plots_path
print(plotter_runbyrun[run].prediction)
print(plotter_runbyrun[run].data)

In [ ]:
#define some useful tools

from matplotlib.lines import Line2D

def Pur(my_plotter,VARIABLE,QUERY,ACCEPTANCE,BINS):
    fig, ax1, stack, labls, exty = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,
        kind="ccncpi0_category",
        draw_sys=False,
        stacksort=3,
        title="my_xaxis",
        bins=BINS,
        range=None,
        detsysdict={'mc':False,'nue':False,'ccpi0':False,'ncpi0':False,'dirt':False,'ncnopi':False,'nccpi':False,'ncpi0':False,'ccpi0':False},
        DETSYSPATH=DETSYSPATH,
        COVMATRIX="",#'cov_matrix_Enu_reco_1eNp.txt',
        ratio=False,
        chisq=False,
    )
    plt.close(fig)
    den = stack[0][-1]+exty
    fig, ax1, stack, labls, exty = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY+' and '+ACCEPTANCE,
        kind="ccncpi0_category",
        draw_sys=False,
        stacksort=3,
        title="my_xaxis",
        bins=BINS,
        range=None,
        detsysdict={'mc':False,'nue':False,'ccpi0':False,'ncpi0':False,'dirt':False,'ncnopi':False,'nccpi':False,'ncpi0':False,'ccpi0':False},
        DETSYSPATH=DETSYSPATH,
        COVMATRIX="",#'cov_matrix_Enu_reco_1eNp.txt',
        ratio=False,
        chisq=False,
    )
    plt.close(fig)
    num = stack[0]
    if num.ndim > 1: num = num[-1]
    return num/den

def Eff(df,var,query,acceptance,bin_edges,absval=False):
    #print acceptance
    bin_centers = 0.5*(bin_edges[1:]+bin_edges[:-1])
    bins = []
    bin_eff = []
    bin_err = []
    for i in range(len(bin_centers)):
        binmin = bin_edges[i]
        binmax = bin_edges[i+1]
        bincut = '%s > %f and %s < %f'%(var,binmin,var,binmax)
        if (absval == True):
            bincut = '(%s > %f and %s < %f) or (%s > -%f and %s < -%f)'%(var,binmin,var,binmax,var,binmax,var,binmin)
        if (acceptance != ''): bincut += ' and %s'%acceptance
        #print bincut
        df_tmp =  df.query(bincut) # cut on bin range for desired var.
        df_sub = df_tmp.query(query) # apply constrain 
        if (df_tmp.shape[0] == 0): continue
        eff = df_sub.shape[0] / float( df_tmp.shape[0] )
        err = np.sqrt( eff*(1-eff)/df_tmp.shape[0] )
        bin_eff.append( eff )
        bin_err.append( err )
        bins.append(bin_centers[i])
        #print 'eff = %.02f @ bin = %.02f'%(eff,bin_centers[i])
    return np.array(bins),np.array(bin_eff),np.array(bin_err)

from scipy.optimize import curve_fit
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))


In [ ]:
#ACCEPTANCE = 'isVtxInFiducial == 1 and npi0 == 1 and ccnc==0 and npion==0 and nu_pdg==14'
#VARR, VART, labl, bin_edges = 'pi0momentum', 'leadPi0_p', 'pi0 momentum', np.array([0.0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.799])
#VARR, VART, labl, bin_edges = 'muon_momentum_ALL', 'Muon_p', 'Muon Momentum', np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5])
#VARR, VART, labl, bin_edges = 'muon_angle', 'Muon_uz', 'Muon CosTheta', np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0])#np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
VARR, VART, labl, bin_edges = 'muon_momentum_exiting', 'Muon_p_exiting', 'Exiting Muon Momentum', np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5])
#VARR, VART, labl, bin_edges = 'muon_theta_exiting', 'Muon_uz_exiting', 'Exiting Muon CosTheta',np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0]) #np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
#VARR, VART, labl, bin_edges = 'muon_momentum_contained_50', 'Muon_p_contained', 'Contained Muon Momentum',np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5])
#VARR, VART, labl, bin_edges = 'muon_theta_contained_50', 'Muon_uz_contained', 'Muon CosTheta', np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0]) #np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
VARR, VART, labl, bin_edges = 'muon_momentum_ALL', 'Muon_p', 'Muon Momentum', np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5]) #np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5])
VARR, VART, labl, bin_edges = 'pi0momentum', 'leadPi0_p', 'pi0 momentum', np.array([0.0,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.799])
#VARR, VART, labl, bin_edges = 'muon_momentum_contained_50', 'Muon_p_contained', 'Contained Muon Momentum',np.array([0.0,0.151,0.34,0.59,0.95,1.45,2.5])
VARR, VART, labl, bin_edges = 'pi0momanglecos', 'leadPi0_uz', 'pi0 CosTheta', np.array([-1.,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0])
#VARR, VART, labl, bin_edges = 'muon_angle', 'Muon_uz', 'Muon CosTheta', np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0])
#VARR, VART, labl, bin_edges = 'muon_angle', 'Muon_uz', 'Muon CosTheta', np.array([-1.,-0.50,0.0,0.30,0.5,0.65,0.78,0.90,1.0])#np.array([-1.,-0.5,0.0,0.28,0.47,0.63, 0.765,0.865,0.935,1.0]) #np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0])
VARR, VART, labl, bin_edges = 'muon_momentum_ALL', 'Muon_p', 'Muon Momentum',np.array([0.0,0.200,0.30,0.44,0.60,0.75,0.95,1.20,2.5]) #np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5])
VARR, VART, labl, bin_edges = 'mu_pi0_angle', 'Muon_pion_angle', r"muon pion opening angle", np.array([-1.,-0.75,-0.50,-0.25,0.0,0.25,0.5,0.75,1.0]) #np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5])
VARR, VART, labl, bin_edges = 'muon_momentum_contained_50', 'Muon_p', r"muon momentum contained",np.array([0.0,0.200,0.30,0.44,0.60,0.75,0.95,1.2,2.5]) #np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5])
VARR, VART, labl, bin_edges = 'muon_momentum_exiting', 'Muon_p', r"muon momentum exiting",np.array([0.0,0.200,0.30,0.44,0.60,0.75,0.95, 1.2, 2.5]) #np.array([0.0,0.200,0.30,0.44,0.60,0.80,1.0,1.30,2.5])

ACCEPTANCE = 'isVtxFiducial_fixed == 1 and npi0 == 1 and ccnc==0 and npion==0 and nu_pdg==14'
#ACCEPTANCE = 'isVtxFiducial_fixed == 1 and npi0 == 1 and ccnc==0 and npion==0 and nu_pdg==14 and truthFiducial==1'
#ACCEPTANCE = 'isVtxFiducial_fixed == 1 and npi0 == 1 and ccnc==0 and npion==0 and nu_pdg==14 and truthFiducial==0'


x = nue.query(ACCEPTANCE+' and '+QUERY_mass)[VARR]
y = nue.query(ACCEPTANCE+' and '+QUERY_mass)[VART]
w = nue.query(ACCEPTANCE+' and '+QUERY_mass)['weightSplineTimesTune']*potw
h2d = np.histogram2d(x,y,weights=w,bins=[bin_edges,bin_edges])

fig = plt.figure(figsize=(6, 6))
sm = h2d[0]
#print(sm)
sm = sm / sm.sum(axis=0)
#print(sm)
#print(sm.sum(axis=0))
#print(sm.sum(axis=1))
plt.imshow(sm,origin='lower')
for i in range(len(sm[0])):
    for j in range(len(sm[0])):
        text = plt.text(j, i, "%.2f"%sm[i, j],ha="center", va="center", color="w")
plt.title('Smearing Matrix\n'+labl)
plt.ylabel("reco "+labl)
plt.xlabel("true "+labl)
plt.xticks(ticks=np.linspace(-0.5,len(bin_edges)-1.5,len(bin_edges)),labels=bin_edges)
plt.yticks(ticks=np.linspace(-0.5,len(bin_edges)-1.5,len(bin_edges)),labels=bin_edges)
plt.colorbar()
plt.tight_layout()
fig.show()
#if savefigs: fig.savefig(save_path + '/xsec-nue-bnb/'+VARR+'_smear_mtrx.pdf', dpi=250)

In [ ]:
DETSYSYES = {'mc':True,'nue':True,'ccpi0':True,'ncpi0':True,'dirt':True,'ncnopi':True,'nccpi':True}
DETSYSNO  = {'mc':False,'nue':False,'ccpi0':False,'ncpi0':False,'dirt':False,'ncnopi':False,'nccpi':False}


In [ ]:
print("ACCEPTANCE=",ACCEPTANCE)
print("VARIABLE=",VARR)
print("VART", VART)
print("QUERY\n", QUERY_mass)

In [ ]:
fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARR,   
    kind="interaction",
    query=QUERY_mass +' and '+ACCEPTANCE,
    draw_sys=False,
    detsysdict=DETSYSYES,
    DETSYSPATH=DETSYSPATH,
    COVMATRIX="",
    ratio=False,
    stacksort=3,
    title=labl,
    bins=bin_edges,
    range=None,
    chisq=False,
)[0:3]
ax1.set_title(TITLE, loc='left')
ax1.set_ylim(0.0, ax1.get_ylim()[1]*1.2)
plt.tight_layout()
fig.show()
#if savefigs: fig.savefig(save_path + '/xsec-nue-bnb/prediction_sig-int_'+VARR+'.pdf', dpi=250)

sigpred = plotter_runbyrun[run].prediction
print('signal-only prediction:',sigpred)

In [ ]:
#let's make a plot in reco space, so that the plotter is internally updated

print(QUERY)

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARR,   
    kind="ccncpi0_category",
    query=QUERY_mass,
    draw_sys=True,
    detsysdict=DETSYSYES,
    DETSYSPATH=DETSYSPATH,
    COVMATRIX="",
    ratio=False,
    stacksort=3,
    title=labl,
    bins=bin_edges,
    range=None,
    chisq=True,
)[0:3]
ax1.set_title(TITLE, loc='left')
ax1.set_ylim(0.0, ax1.get_ylim()[1]*1.2)
plt.tight_layout()
fig.show()
#if savefigs: fig.savefig(save_path + '/xsec-nue-bnb/prediction_'+VARR+'.pdf', dpi=250)

pred = plotter_runbyrun[run].prediction
obsv = plotter_runbyrun[run].data
print('total prediction:',pred)
print('observed data:',obsv)

In [ ]:
gcov = plotter_runbyrun[run].sys_err_with_resp_func("weightsGenie", sname, VARR, VART, QUERY_mass, ACCEPTANCE, None, bin_edges, "weightSplineTimesTune", maxUniv=True)
print('genie\n',gcov)
ucov = plotter_runbyrun[run].sys_err_unisim_with_resp_func(sname, VARR, VART, QUERY_mass, ACCEPTANCE, None, bin_edges)
print('unsim\n',ucov)
fcov = plotter_runbyrun[run].sys_err("weightsFlux", VARR, QUERY_mass, None, bin_edges, "weightSplineTimesTune",maxUniv=True)
print('flux\n',fcov)
rcov = plotter_runbyrun[run].sys_err("weightsReint", VARR, QUERY_mass, None, bin_edges, "weightSplineTimesTune",maxUniv=True)
print('reint\n',rcov)


In [ ]:
scov = plotter_runbyrun[run].cov_mc_stat
print('mcstat\n',scov)
dcov = plotter_runbyrun[run].cov_mc_detsys
print('detector\n',dcov)

In [ ]:
# beam exposure 2% of nu events
# number of targets 1% of nu events

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARR,   
    kind="ccncpi0_category",
    query=QUERY_mass +' and extdata==0 and bnbdata==0',
    draw_sys=False,
    detsysdict=DETSYSNO,
    DETSYSPATH=DETSYSPATH,
    COVMATRIX="",
    ratio=False,
    stacksort=3,
    title=labl,
    bins=bin_edges,
    range=None,
    chisq=False,
)[0:3]
ax1.set_title(TITLE, loc='left')
ax1.set_ylim(0.0, ax1.get_ylim()[1]*1.2)
plt.tight_layout()
fig.show()

nupred = plotter_runbyrun[run].prediction
print('nu-only prediction:',nupred)

pcov = np.empty([len(nupred), len(nupred)])
ncov = np.empty([len(nupred), len(nupred)])
for i in range(0,len(nupred)):
    for j in range(0,len(nupred)):
        pcov[i,j] = 0.02*0.02*nupred[i]*nupred[j]
        ncov[i,j] = 0.01*0.01*nupred[i]*nupred[j]
print('POT\n',pcov)
print('Ntargets\n',ncov)

In [ ]:
print('pred=',pred)

totcov = gcov+ucov+fcov+rcov+scov+dcov+pcov+ncov
print('tot cov\n',totcov)

fig = plt.figure(figsize=(8,8))
plt.imshow(totcov,origin='lower')
for i in range(len(totcov[0])):
    for j in range(len(totcov[0])):
        text = plt.text(j, i, "%.2f"%totcov[i, j],ha="center", va="center", color="w")
plt.title('Covariance Matrix\n'+labl)
plt.ylabel(labl)
plt.xlabel(labl)
plt.xticks(ticks=np.linspace(-0.5,len(bin_edges)-1.5,len(bin_edges)),labels=bin_edges)
plt.yticks(ticks=np.linspace(-0.5,len(bin_edges)-1.5,len(bin_edges)),labels=bin_edges)
plt.colorbar()
plt.tight_layout()
fig.show()
#if savefigs: fig.savefig(save_path + '/xsec-nue-bnb/'+VARR+'_cov.pdf', dpi=250)

fig = plt.figure(figsize=(6,6))
totfcov = totcov/(pred*pred)
plt.imshow(totfcov,origin='lower')
for i in range(len(totfcov[0])):
    for j in range(len(totfcov[0])):
        text = plt.text(j, i, "%.3f"%totfcov[i, j],ha="center", va="center", color="w")
plt.title('Fractional Covariance Matrix\n'+labl)
plt.ylabel(labl)
plt.xlabel(labl)
plt.xticks(ticks=np.linspace(-0.5,len(bin_edges)-1.5,len(bin_edges)),labels=bin_edges)
plt.yticks(ticks=np.linspace(-0.5,len(bin_edges)-1.5,len(bin_edges)),labels=bin_edges)
plt.colorbar()
plt.tight_layout()
fig.show()
#if savefigs: fig.savefig(save_path + '/xsec-nue-bnb/'+VARR+'_frac_cov.pdf', dpi=250)

tdiag = np.sqrt(np.diag(totcov)) 
print('diag err=',tdiag)
tfdiag = tdiag/pred
print('frac diag err=',tfdiag)

In [ ]:
den = pred #sigpred
bincenters = 0.5*(bin_edges[1:]+bin_edges[:-1])
gfdiag = np.sqrt(np.diag(gcov))/den
ffdiag = np.sqrt(np.diag(fcov))/den
rfdiag = np.sqrt(np.diag(rcov))/den
sfdiag = np.sqrt(np.diag(scov))/den
dfdiag = np.sqrt(np.diag(dcov))/den
pfdiag = np.sqrt(np.diag(pcov))/den
nfdiag = np.sqrt(np.diag(ncov))/den

print('GENIE',gfdiag)
print('Flux',ffdiag)
print('Reint',rfdiag)
print('MCStat',sfdiag)
print('DetSyst',dfdiag)
print('POT',pfdiag)
print('NTargets',nfdiag)

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111)
ax.hist(bincenters,bins=bin_edges,weights=gfdiag,histtype='step',label='GENIE')
ax.hist(bincenters,bins=bin_edges,weights=ffdiag,histtype='step',label='Flux')
ax.hist(bincenters,bins=bin_edges,weights=rfdiag,histtype='step',label='Reint')
ax.hist(bincenters,bins=bin_edges,weights=sfdiag,histtype='step',label='MCStat')
ax.hist(bincenters,bins=bin_edges,weights=dfdiag,histtype='step',label='DetSyst')
#ax.hist(bincenters,bins=bin_edges,weights=xfdiag,histtype='step',label='DetSyst+MCStat+Reint')
ax.hist(bincenters,bins=bin_edges,weights=pfdiag,histtype='step',label='POT')
ax.hist(bincenters,bins=bin_edges,weights=nfdiag,histtype='step',label='NTargets')
ax.hist(bincenters,bins=bin_edges,weights=tfdiag,histtype='step',label='Total',color='black')
ax.hist(bincenters,bins=bin_edges,weights=np.sqrt(pred)/den,histtype='step',label='Data Stat, exp.',color='black',ls='--')
handles, labels = ax.get_legend_handles_labels()
new_handles = [Line2D([], [], c=h.get_edgecolor(), linestyle=h.get_linestyle()) for h in handles]
plt.legend(handles=new_handles, labels=labels,loc=2,ncol=3,frameon=False)
#plt.legend(loc=2,ncol=3,frameon=False)
plt.xlabel(labl)
plt.ylabel('Fractional Uncertainty on Total Predicted Events')
plt.ylim(0,1.0)
plt.tight_layout()
fig.show()
#if savefigs: fig.savefig(save_path + '/xsec-nue-bnb/'+VARR+'_frac_unc.pdf', dpi=250)

In [ ]:
## response matrix

rm, xb, yb = plotter_runbyrun[run].ResponseMatrix(nue,ACCEPTANCE,QUERY_mass,\
                                                  VART,VARR,bin_edges,'weightSplineTimesTune',potw)
print('rm\n',rm)
fig = plt.figure(figsize=(6, 6))
plt.imshow(np.transpose(rm),origin='lower')
#print(np.shape(rm)[0])
for i in range(np.shape(rm)[0]):
    for j in range(np.shape(rm)[1]):
        text = plt.text(i, j, "%.3f"%rm[i, j],ha="center", va="center", color="w")
plt.title('Response Matrix\n'+labl)
plt.xlabel('true '+labl)
plt.ylabel('reco '+labl)
plt.xticks(ticks=np.linspace(-0.5,len(xb)-1.5,len(xb)),labels=xb)
plt.yticks(ticks=np.linspace(-0.5,len(yb)-1.5,len(yb)),labels=yb)
plt.colorbar()
plt.tight_layout()
fig.show()
#if savefigs: fig.savefig(save_path + '/xsec-nue-bnb/'+VARR+'_resp_mtrx.pdf', dpi=250)

In [ ]:
#closure test for response matrix
tg = nue.query(ACCEPTANCE)[VART]
w1 = nue.query(ACCEPTANCE)['weightSplineTimesTune']*potw
tv = np.histogram(tg,weights=w1,bins=bin_edges)

print("truth-level signal, before selection:",tv[0])
print("response matrix:\n",rm)

rp = rm.dot(tv[0])

print("reco signal from response matrix: ",rp)

ro = nue.query(ACCEPTANCE+' and '+QUERY_mass)[VARR]
w2 = nue.query(ACCEPTANCE+' and '+QUERY_mass)['weightSplineTimesTune']*potw
tr = np.histogram(ro,weights=w2,bins=bin_edges)
print("reco signal straight from ntuples:",tr[0])

In [ ]:
bin_edges

In [ ]:
genieuntuned = False
bkgonly = False
sigonly = False
#replace QUERY to allow for data
FDQUERY = QUERY_mass.replace("bnbdata == 0","bnbdata >= 0")
if bkgonly: FDQUERY = FDQUERY+' and ~('+ACCEPTANCE+')'#test if background agrees
if sigonly: FDQUERY = FDQUERY+' and ('+ACCEPTANCE+')'#signal only
print(FDQUERY)

FDTITLE = TITLE.replace("Blind","NuWro Fake Data")
if bkgonly: FDTITLE += " - Bkg Only"
if sigonly: FDTITLE += " - Sig Only"
genieweight = "weightSplineTimesTune"
if genieuntuned: genieweight = "weightSpline"

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARR,   
    kind="ccncpi0_category",
    query=QUERY_mass,
    genieweight=genieweight,
    draw_sys=True,
    detsysdict=DETSYSYES,
    DETSYSPATH=DETSYSPATH,
    COVMATRIX="",
    ratio=False,
    stacksort=3,
    title=labl,
    bins=bin_edges,
    range=None,
    chisq=True,
)[0:3]

# Some cosmetics by hand to set the position of the chi2
for a in ax1.get_children(): 
    if len(a.findobj(match=plt.Text))==0: continue
    txt = a.findobj(match=plt.Text)[0]
    if 'n.d.f.' not in txt.get_text(): continue
    #print(txt.get_text())
    if VARR == 'cos_trk_theta' or VARR == 'cos_shr_theta':
        txt.set_x(0.20)
    if VARR == 'protonenergy_corr':
        txt.set_x(0.80)
    txt.set_y(0.38)
    
ax1.set_title(FDTITLE, loc='left')
ax1.set_ylim(0.0, ax1.get_ylim()[1]*1.5)
plt.tight_layout()
fig.show()
    
#print(plotter_runbyrun[run].data)
print('data:', plotter_runbyrun[run].data)
if bkgonly==False and sigonly==False:
    data = plotter_runbyrun[run].data

In [ ]:
genieuntuned = False
bkgonly = True
sigonly = False

#replace QUERY to allow for data
FDQUERY = QUERY_mass.replace("bnbdata == 0","bnbdata >= 0")
if bkgonly: FDQUERY = QUERY_mass+' and ~('+ACCEPTANCE+')'#test if background agrees
if sigonly: FDQUERY = FDQUERY+' and ('+ACCEPTANCE+')'#signal only
#print(FDQUERY)

FDTITLE = TITLE.replace("Blind","NuWro Fake Data")
if bkgonly: FDTITLE += " - Bkg Only"
if sigonly: FDTITLE += " - Sig Only"
genieweight = "weightSplineTimesTune"
if genieuntuned: genieweight = "weightSpline"

fig, ax1, ax2 = plotter_runbyrun[run].plot_variable(
    VARR,   
    kind="ccncpi0_category",
    query=FDQUERY,
    genieweight=genieweight,
    draw_sys=True,
    detsysdict=DETSYSYES,
    DETSYSPATH=DETSYSPATH,
    COVMATRIX="",
    ratio=False,
    stacksort=3,
    title=labl,
    bins=bin_edges,
    range=None,
    chisq=True,
)[0:3]
#print(plotter_runbyrun[run].data)
print('background data:', plotter_runbyrun[run].prediction)
databs = data-plotter_runbyrun[run].prediction

In [ ]:
print(np.diag(data))

# covariance matrix for fake data
# need to add back flux, reint, detsyst for real data!!!!
Ftotcov = np.diag(data)+gcov+ucov+fcov+rcov+scov+dcov+pcov+ncov
#totcov = gcov+ucov+fcov+rcov+scov+dcov+pcov+ncov
print(Ftotcov)

In [ ]:
databs

In [ ]:
from ROOT import TMatrixD, TH1F, TH2F, TFile 
#theFile = TFile(rootfile, "recreate")
#InFile = TFile(“MOM_genie_v3_MC.root”,“recreate”)
#TFile *hfile = new TFile(“MOM_genie_v3_MC.root”)
outHistFile = TFile.Open('muon_mom_exiting_Data_Test_full_bin_.root',"RECREATE")
outHistFile.cd()
NBINS = 8
#bin_edges = np.array([0.000, 0.100, 0.150, 0.200, 0.300, 0.400, 0.500, 0.600, 0.799])
#bin_edges = np.array([-1.000, -0.75,-0.500, -0.250, 0.000, 0.250, 0.500, 0.750, 1.000])
#bin_edges = np.array([0.000, 0.151, 0.340, 0.590, 0.950, 1.450, 2.500])
bin_edges = np.array([0.0,0.200,0.30,0.44,0.60,0.75,0.95, 1.2, 2.5]) 
#bin_edges = np.array([-1.000, -0.500, 0.000, 0.300, 0.500, 0.650, 0.780, 0.900, 1.000])
prior_true_signal = TH1F("prior_true_signal", "prior true signal", NBINS, bin_edges)
data_signal = TH1F("data_signal", "Measured signal", NBINS, bin_edges)
respmat = TH2F("respmat", "Response Matrix", NBINS, bin_edges, NBINS, bin_edges)
covmat =  TH2F("covmat", "Covariance Matrix", NBINS, bin_edges, NBINS, bin_edges)
#genie_true_signal = TH1F("genie_true_signal", "genie true signal", NBINS, bin_edges)

for i in range(0, NBINS):
    prior_true_signal.SetBinContent(i+1, tv[0][i])
    data_signal.SetBinContent(i+1, databs[i] )
    #genie_true_signal.SetBinContent(i+1, tv[0][i])
    
for row in range(0,NBINS):
    for column in range(0,NBINS):
        respmat.SetBinContent(row+1, column+1, rm[row, column])
        covmat.SetBinContent(row+1, column+1, Ftotcov[row, column])

prior_true_signal.Write()
data_signal.Write()
respmat.Write()
covmat.Write()
#genie_true_signal.Write()

# write and close
outHistFile.Close()